# Corporate Decarbonization Research
---

### Import modules

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

### Table of Contents

**[Section 1) Loading and Cleaning](#section-1-loading-and-cleaning)**
- Loading the Raw Data
- Helper Functions
- More Data Cleaning

**[Section 2) EDA and Summary Statistics](#section-2-eda-and-summary-statistics)**
- Data Dictionary
- Cross-sector EDA

**[Section 3) Visualizations](#section-3-visualizations)**
- CDP Submissions
- Trends by Sector Charts
- Scatter Plots
- Company Emissions Charts
- Net-Zero Charts

**[Section 4) Conclusion](#section-4---conclusion)**
- What we learned
- Next Steps
- Collaboration efforts

---

# SECTION 1) Loading and Cleaning

## Loading the Raw Data

*In this section of the notebook, I will be importing our data and performing some preliminary cleaning/standardizing of our datasets to create separate DataFrames for each sector and an aggregated DataFrame containing all sectors. More data cleaning will be performed after the **Helper Functions** section.*

In [ ]:
# load in data

foodag = pd.read_csv('data/foodag.csv')
energy = pd.read_csv('data/energy.csv')
auto = pd.read_csv('data/auto.csv')
tech = pd.read_csv('data/tech.csv')


In [ ]:
# Add sector columns to concatenate df's

foodag['SECTOR'] = ['Food & Agriculture'] * len(foodag['COMPANY NAME'])
energy['SECTOR'] = ['Energy'] * len(energy['COMPANY NAME'])
auto['SECTOR'] = ['Auto'] * len(auto['COMPANY NAME'])
tech['SECTOR'] = ['Tech'] * len(tech['COMPANY NAME'])
foodag.shape, energy.shape, auto.shape, tech.shape


In [ ]:
# clean tech to standardize columns across all df's, shorten CI column name

tech = tech.rename(columns={'CARBON INTENSITY\n(Scope 1 & 2 g CO2e/ $ Sales) \ncalculated' :
'CARBON INTENSITY\n(Scope 1 & 2 g CO2e / $ Sales)'})

df_array = [foodag, energy, auto, tech]
for df in df_array:
    df.rename(columns={'CARBON INTENSITY\n(Scope 1 & 2 g CO2e / $ Sales)': 'CARBON INTENSITY'}, inplace=True)


In [ ]:
# concatenate dataframes together

sectors = pd.concat([foodag, energy, auto, tech])
sectors = sectors.drop(columns='SCOPE 1 + SCOPE 2 EMISSIONS')
sectors.shape

### *To access a dataset containing all sectors, use the `sectors` DataFrame. To acccess individual sectors, choose from `foodag`, `tech`, `auto`, and `energy`.*

### `sectors`

In [ ]:
# show sectors

sectors.head()

### Example Sector DataFrame: `foodag`

In [ ]:
# show food and ag

foodag.head()

In [ ]:
# scratch work code cell, replace elipses with any code you'd like to try

---

## Helper Functions

*This section will contain any functions used to generate graphs, standardize columns, etc.*

In [ ]:
# convert revenue's to USD

def toUSD(i):
    dic = {
    'AUD': 0.75,   # 1 AUD to USD
    'CAD': 0.78,   # 1 CAD to USD
    'CHF': 1.09,   # 1 CHF to USD
    'DKK': 0.15,   # 1 DKK to USD
    'EUR': 1.16,   # 1 EUR to USD
    'GBP': 1.37,   # 1 GBP to USD
    'JPY': 0.009,  # 1 JPY to USD
    'KRW': 0.0009, # 1 KRW to USD
    'NOK': 0.11,   # 1 NOK to USD
    'RUB': 0.014,  # 1 RUB to USD
    'SEK': 0.11,   # 1 SEK to USD
    'RMB': 7.29,   # 1 RMB to USD
    'TWD' : 0.03,  # 1 TWD to USD
    'USD': 1.0       # No Change    
    }
    if (not pd.isna(i[0])) or (not pd.isna(i[1])):
        return i[0] * dic[i[1]] # The revenue multiplied by the corresponding USD conversion rate

# sectors['Revenue (USD)'] = sectors.apply(toUSD, 'TOTAL REVENUE (miillion $)')  # --- use this code to apply the function in the revenue column when ready

In [ ]:
# Function for creating column with [Currency, Revenue]

def currAndRev(df):
    new_col = []
    for i in np.arange(df.shape[0]):
        new_col.append([df['CURRENCY'].values[i], df['TOTAL REVENUE (miillion $)'].values[i]])
    df['REVENUE IN USD'] = new_col

# currAndRev(sectors)


In [ ]:
# graph sector summary charts using matplotlib

def graphCoEmissions(df, co_name, years_arr):
    # format plots and add right axis for CI
    fig, ax1 = plt.subplots()
    ax2 = ax1.twinx()

    # create x axis array
    years = df.loc[df['COMPANY NAME']==co_name, 'YEAR']

    # create bar chart stacks and CI array
    scope1 = df.loc[df['COMPANY NAME']==co_name, 'SCOPE 1']
    scope2 = df.loc[df['COMPANY NAME']==co_name, 'SCOPE 2 (location-based)']
    scope3 = df.loc[df['COMPANY NAME']==co_name, 'SCOPE 3']
    ci = df.loc[df['COMPANY NAME']==co_name, 'CARBON INTENSITY']

    # plot bar chart
    b1 = ax1.bar(years, scope1, color='navy', label='Scope 1')
    b2 = ax1.bar(years, scope2, bottom=scope1, color='steelblue', label='Scope 2')
    b3 = ax1.bar(years, scope3, bottom=scope1+scope2, color='lightsteelblue', label='Scope 3')

    # plot CI
    l1 = ax2.plot(years, ci, color = 'black', marker = 'o',label='CI')

    # finish formatting plots
    ax1.set_xticks(years_arr)
    ax1.set_ylim(0, max(scope1+scope2+scope3)+9000000)
    ax2.set_ylim(0, max(ci)+20)

    # combine and add legend
    # lines = [b1, b2, b3]
    # labels = ['SCOPE 1','SCOPE 2','SCOPE 3' ]
    # ax1.legend(lines, labels, loc='lower left', bbox_to_anchor=(1, 1))
    # ax2.legend(loc=0)

    lines, labels = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax2.legend(lines + lines2, labels + labels2, loc='lower left', bbox_to_anchor=(1, 1))

    # add labels and titles
    plt.suptitle('Annual GHG Emissions', fontsize=14, fontweight='bold')
    plt.title(co_name, fontsize=10)
    ax1.set_xlabel('Year')
    ax1.set_ylabel('Carbon Emissions (metric ton CO2e)')
    ax2.set_ylabel('Carbon Intensity (g CO2e / $ million)')

In [ ]:
# graph sector summary charts using plotly
import plotly.graph_objects as go

def graphCoEmissionsPlotly(df, co_name, years_arr):
    # format plots and add right axis for CI
    # fig, ax1 = plt.subplots()
    # ax2 = ax1.twinx()

    # create x axis array
    years = df.loc[df['COMPANY NAME']==co_name, 'YEAR']

    # create bar chart stacks and CI array
    scope1 = df.loc[df['COMPANY NAME']==co_name, 'SCOPE 1']
    scope2 = df.loc[df['COMPANY NAME']==co_name, 'SCOPE 2 (location-based)']
    scope3 = df.loc[df['COMPANY NAME']==co_name, 'SCOPE 3']
    ci = df.loc[df['COMPANY NAME']==co_name, 'CARBON INTENSITY']


     # create stacked bar chart traces
    trace_scope1 = go.Bar(x=years, y=scope1, name='Scope 1', marker=dict(color='rgba(0, 0, 255, 0.7)'))
    trace_scope2 = go.Bar(x=years, y=scope2, name='Scope 2', marker=dict(color='rgba(0, 255, 0, 0.7)'))
    trace_scope3 = go.Bar(x=years, y=scope3, name='Scope 3', marker=dict(color='rgba(255, 0, 0, 0.7)'))

    # create line chart trace for CI
    trace_ci = go.Scatter(x=years, y=ci, mode='markers+lines', name='Carbon Intensity', yaxis='y2',
                          marker=dict(color='rgba(255, 165, 0, 0.7)'))

    # combine traces into data list
    data = [trace_scope1, trace_scope2, trace_scope3, trace_ci]

    # create layout
    layout = go.Layout(
        title=dict(text=co_name+' - Annual GHG Emissions', x=0.5),
        xaxis=dict(tickvals=years_arr, title='Year'),
        yaxis=dict(title='Carbon Emissions (metric ton CO2e)', range=[0, max(scope1+scope2+scope3)+9000000]),
        yaxis2=dict(title='Carbon Intensity (g CO2e / $ million)', overlaying='y', side='right', range=[0, max(ci)+20]),
        barmode='stack',
        showlegend=True,
        height=650,
        width=800,
        margin=dict(r=1.2)
    )

    # create figure
    fig = go.Figure(data=data, layout=layout)

    # show the figure
    fig.show()

In [ ]:
# graph sector totals by company and emissions type

def graph_CosInSectorTotals(df, year, bool=False):
    # define the sector
    sector = df['SECTOR'].unique()[0]

    # filer DataFrame for just the provided year
    data_year = df.loc[df['YEAR']==year,:]

    # plot stacked bar chart
    sns.barplot(x='SCOPE 1', y='COMPANY NAME', data=data_year, color='orange', orient='h', label='Scope 1')
    sns.barplot(x='SCOPE 2 (location-based)', y='COMPANY NAME', data=data_year, color='navy', orient='h', label='Scope 2')
    if bool==False:
        sns.barplot(x='SCOPE 3', y='COMPANY NAME', data=data_year, color='pink', orient='h', label='Scope 3')
    
    # format axes and titles
    plt.xlabel('GHG emissions (metric tons CO2e)')
    plt.ylabel('Company Name')
    if bool==True:
        plt.title('Scope 1 + Scope 2 GHG Emissions in '+str(year)+' - ' +str(sector))
    else:
        plt.title('Total GHG Emissions in '+str(year)+' - ' +str(sector))

In [ ]:
# graph carbon intensity as a function of time of each company in a sector using adjusted CI values

def ci_OverTime(sector):

    companies = sectors.loc[sectors['SECTOR']==sector, 'COMPANY NAME'].unique()
    years = ['2018','2019', '2020', '2021', '2022']

    for co in companies:
        ci = sectors[sectors['COMPANY NAME'] == co]["Adjusted CI"]
        plt.plot(years, ci, label = co, marker='o', markersize=5)
    plt.legend(loc = 'upper right', fontsize = 'xx-small')
    plt.xlabel('Year')
    plt.ylabel('Carbon Intensity (mt $CO_{2}e$ / mill USD)')
    plt.title(sector +  ' Carbon Intensity Over Time (2018-2022)');

---

## More Data Cleaning

*In this section, I am performing the data cleaning which require my **helper functions** to execute.* Make sure that the previous section has been run before proceeding.

In [ ]:
# apply and create revenue in USD column

rev_curr = [[sectors['TOTAL REVENUE (miillion $)'].values[i], sectors['CURRENCY'].values[i]] for i in range(sectors.shape[0])]
sectors['REVENUE W/ CURR'] = rev_curr

# create revenue in USD column and drop revenue w/ curr column

sectors['REVENUE IN USD'] = sectors['REVENUE W/ CURR'].apply(toUSD)
sectors = sectors.drop(columns='REVENUE W/ CURR')

In [ ]:
# creating adjusted CI column after converting ALL revenues to USD

adjusted_ci = ((sectors['SCOPE 1'] + sectors['SCOPE 2 (location-based)']) / sectors['REVENUE IN USD']).round(2)
sectors['Adjusted CI'] = adjusted_ci

In [ ]:
# Replace 'Submit to CDP' & 'SBTi Target Set?' values with True and False for performing qualitative data analysis

sectors['SUBMIT TO CDP'] = sectors['SUBMIT TO CDP'].replace({
    'Yes' : True,
    'No' : False
})

sectors['SBTi TARGET SET?'] = sectors['SBTi TARGET SET?'].replace({
    'Yes' : True,
    'No' : False
})

In [ ]:
sectors.head()

In [ ]:
load_sectors = sectors.to_csv('data/sectors.csv', index=False)
load_sectors

---

# SECTION 2) EDA and Summary Statistics

## Data Dictionary

| Column Name | Definition | Data Type |
|--------------|:-----:|-----------:|
| Company Name | Name of the company | object |
| Year | Corresponding year of emissions data | int |
| Scope 1 | The company's direct GHG emissions, given in metric tons of carbon dioxide equivalence | float |
| Scope 2 (location-based) | The company's indirect (location-based) GHG emissions, given in metric tons of carbon dioxide equivalence | float |
| Scope 3 | The company's indirect GHG emissions that come from its value chain, given in metric tons of carbon dioxide equivalence | float |
| Total Emissions | Sum of Scope 1, 2, and 3 emissions | float |
| Total Revenue (million $) | The company's total revenue for a given year | float |
| Currency | The currency of the company's total revenue | object |
| Carbon Intensity | Total scope 1 and scope 2 (location-based) emissions per unit of the company's revenue | float |
| Country of Origin | The company's origin country | object |
| Submit to CDP | Boolean indicator of if the company submitted to CDP for the reporting year (True/False) | object |
| SBTi Target Set? | Boolean indicator of if the company has targets approved by SBTi | object |
| SBTI Commitments | A company's net-zero and near-term targets status | object | 
| Net-Zero Targets | A company's net-zero or emissions reductions target(s) | object | 
| Sector | The industry sector of the company | object |



## Cross-Sector Exploratory Data Analysis (EDA)

*This section is dedicated to data analysis through visualizations and observing general trends in the data across sectors.*

### Looking at average scope, average total emissions, average, total revenue, and average CI by sector.

The GroupBy object created below, sorted in descending order by `TOTAL EMISSIONS`, shows us that **energy** is the highest-emitting sector while **tech** comes last out of our four sectors.

In [ ]:
# Average scope values by sector in descending order of average total emissions

avg_sectors = sectors.groupby('SECTOR').mean(numeric_only=True).round(0).sort_values(by='TOTAL EMISSIONS', ascending=False)[['SCOPE 1', 'SCOPE 2 (location-based)', 'SCOPE 3',  'TOTAL EMISSIONS']]
avg_sectors

In [ ]:
# Average Corporate Revenues converted to USD

sectors.groupby('SECTOR').mean(numeric_only=True).sort_values(by='REVENUE IN USD', ascending=False).round(0)[['REVENUE IN USD']]


### A rough look at highest-emitting companies on average

This GroupBy object shows us the highest-emitting companies on average, sorted by average `TOTAL EMISSIONS`. 

*NOTE: This is not a completely reflective ranking because it doesn't take into account the companies that don't report certain scope values (indicated by NaN).*

In [ ]:
sectors.groupby('COMPANY NAME').mean(numeric_only=True).drop(columns=['YEAR']).sort_values(by='TOTAL EMISSIONS', ascending=False)

### Average Emissions by Country of Origin

Now let's take a look at the average emissions by country of origin, still sorting by average `TOTAL EMISSIONS` in descending order.

*NOTE: I am still working on figuring out how to differentate companies with more than one country of origin.*

In [ ]:
avg_ByCountry = sectors.groupby('COUNTRY OF ORIGIN').mean(numeric_only=True).drop(columns=['YEAR', 'TOTAL REVENUE (miillion $)']).round(0).sort_values(by='TOTAL EMISSIONS', ascending=False)

avg_ByCountry

The first row corresponds to **Shell**, last row corresponds to **Molson Coors**.

In [ ]:
# scratch work code cell, replace elipses with any code you'd like to try

...

---

# SECTION 3) Visualizations

### CDP Submission: Who Reported to CDP?

The purpose of this section is to view the proportion of companies across all sectors who reported to CDP.

In [ ]:
# Proportion of CDP submissions over number of reports

submission_percentage = (sum(sectors['SUBMIT TO CDP'] == True) / sectors.shape[0]) * 100

print('Across all sectors, ' + str(round(submission_percentage,0))+ '% of companies submitted to cdp')

Now we know 70% of our data comes from CDP submissions. To compare CDP submissions per sector visually, I created a **count plot** using `sectors`.

In [ ]:
# Plot counts of submissions

sns.countplot(x='SECTOR', hue='SUBMIT TO CDP', data=sectors)
plt.xlabel('Sector')
plt.ylabel('Count')
plt.title('Count of CDP Submissions by Sector')

### CDP Submissions: How do submission trends look over time?

In [ ]:
# Visualizing CDP Submissions Over TIme

x = np.arange(2017,2023)
y = sectors.groupby('YEAR').count()['SUBMIT TO CDP']

fig, ax = plt.subplots()
plt.bar(x, y)
plt.suptitle('Count of CDP Submissions Over Time (2017-2022)', fontweight='bold')
plt.xlabel('Year')
plt.ylabel('Submissions Counts')
bars = ax.bar(x, y)
ax.bar_label(bars)
plt.show();

### Scatter Plots: 
### Scope 1 vs. Scope 2 (location-based) Across All Sectors

This scatterplot reflects the significant difference in emissions emitted by the **energy** sector compared with any other sector. Aside from **energy**, **tech** seems to have relatively higher scope 2 values. Hover over a data point to retrieve information on the represented company and the actual scope values associated. To take a closer look at a certain interval, drag your mouse across the desired interval. To reset the plot, run the code cell again. Any plot generated through Plotly can also be downloaded as a png by clicking on the camera icon on upper right portion of the plot. 

In [ ]:
# scope 1 vs scope 2 (location-based)

fig = px.scatter(sectors, x='SCOPE 1', y='SCOPE 2 (location-based)', color='SECTOR', hover_data=['COMPANY NAME', 'YEAR'])
fig.update_layout(title='Scope 1 vs. Scope 2 Across All Sectors')
fig.show()

### Scope 2 (location-based) vs. Scope 3


In [ ]:
# scope 2 (location-based) vs. scope 3 

fig = px.scatter(sectors, x='SCOPE 2 (location-based)', y='SCOPE 3', color='SECTOR', hover_data=['COMPANY NAME'])
fig.update_layout(title='Scope 2 vs. Scope 3 Across All Sectors')
fig.show()

### Scope 1 vs. Scope 3 

In [ ]:
# scope 1 vs. scope 3

fig = px.scatter(sectors, x='SCOPE 1', y='SCOPE 3', color='SECTOR', hover_data=['COMPANY NAME'])
fig.update_layout(title='Scope 1 vs. Scope 3 Across All Sectors')
fig.show()

In [ ]:
# scratch work code cell, replace elipses with any code you'd like to try

...

---

## Sector Charts [WORK IN PROGRESS]

*The goal of this section is to generate charts like the ones in the **Cross-Sector** tab of the **emissions** Google Sheet.*

### Carbon Intensity Over Time by Sector

In [ ]:
# Food & Ag
ci_OverTime('Food & Agriculture')

### Total GHG Emissions Over Time by Sector

In [ ]:
energy_totals = energy.groupby('YEAR').sum(numeric_only=True)
energy_totals['TOTAL EMISSIONS']

years=[2018,2019,2020,2021,2022]
total_emissions = energy_totals['TOTAL EMISSIONS'].values / 1000000

fig, ax = plt.subplots()

ax.bar(years, total_emissions, color='navy')
plt.xlabel('Year')
plt.ylabel('GHG Emissions (million mt $CO_{2}e$)')
plt.title('Energy Sector', fontsize=10)
plt.suptitle('Total GHG Emissions (2018-2022)', fontsize=14, fontweight='bold')
bars = ax.bar(years, total_emissions)
ax.bar_label(bars)
plt.show();

In [ ]:
# Energy
ci_OverTime('Energy')

In [ ]:
# Tech
ci_OverTime('Tech')

<span style="color:red">TASK: Debug auto (function won't work for auto sector).</span>

In [ ]:
# Auto
# ci_OverTime('Auto')

---

## Company Charts by Sector

*The charts generated below follow the format of the charts we generated in Google Sheets. Make sure to run the **Helper Functions** section in order to generate this section properly.*

### Food & Agriculture Emissions Charts

In [ ]:
# graph charts for every company

yrs1 = [2017, 2018, 2019, 2020]
yrs2 = [2018, 2019, 2020, 2021]
yrs3 = [2018, 2019, 2020, 2021, 2022]
yrs4 = [np.arange(2017, 2023)]

for company in foodag['COMPANY NAME'].unique():
    num_years = foodag.loc[foodag['COMPANY NAME']==company, 'YEAR'].shape[0]
    if num_years == 4:
        graphCoEmissions(foodag, company, yrs2)
    else:
        graphCoEmissions(foodag, company, yrs3)

In [ ]:
# another way to plot charts
# import plotly.graph_objects as go

# for company in foodag['COMPANY NAME'].unique():
#     num_years = foodag.loc[foodag['COMPANY NAME']==company, 'YEAR'].shape[0]
#     if num_years == 4:
#         graphCoEmissionsPlotly(foodag, company, yrs2)
#     else:
#         graphCoEmissionsPlotly(foodag, company, yrs3)


### Energy Emissions Charts

<span style="color:red">TASK: Exclude years with no data.</span>

In [ ]:
# graph energy charts

for company in energy['COMPANY NAME'].unique():
    num_years = energy.loc[energy['COMPANY NAME']==company, 'YEAR'].shape[0]
    if num_years == 4:
        graphCoEmissionsPlotly(energy, company, yrs2)
    else:
        graphCoEmissionsPlotly(energy, company, yrs3)

### Tech Emissions Charts

In [ ]:
# graph tech charts

for company in tech['COMPANY NAME'].unique():
    num_years = tech.loc[tech['COMPANY NAME']==company, 'YEAR'].shape[0]
    if num_years == 4:
        graphCoEmissions(tech, company, yrs2)
    else:
        graphCoEmissions(tech, company, yrs3)

### Auto Emissions Charts

In [ ]:
for company in auto['COMPANY NAME'].unique():
    num_years = auto.loc[auto['COMPANY NAME']==company, 'YEAR'].shape[0]
    if num_years == 4:
        graphCoEmissionsPlotly(auto, company, yrs2)
    else:
        graphCoEmissionsPlotly(auto, company, yrs3)

In [ ]:
# scratch work code cell, replace elipses with any code you'd like to try

...

---

## Net-Zero Goals

I'd like to create simple visuals depicting net-zero goals for each firm

In [ ]:
sectors['NET-ZERO TARGETS'].isna().sum()

---

# Section 4) Conclusion

maybe link paper here?

## What We Learned


## Next Steps

## Collaboration Efforts

Contribute to our live database by filling out this Google Form and report your own emissions data!

---

In [ ]:
e = energy.groupby('YEAR').sum(numeric_only=True)
e['TOTAL EMISSIONS']


In [ ]:
energy_totals = energy.groupby('YEAR').sum(numeric_only=True)
energy_totals['TOTAL EMISSIONS']

years=[2018,2019,2020,2021,2022]
total_emissions = energy_totals['TOTAL EMISSIONS'].values / 1000000

fig, ax = plt.subplots()

ax.bar(years, total_emissions, color='navy')
plt.xlabel('Year')
plt.ylabel('GHG Emissions (million mt $CO_{2}e$)')
plt.title('Energy Sector', fontsize=10)
plt.suptitle('Total GHG Emissions (2018-2022)', fontsize=14, fontweight='bold')
bars = ax.bar(years, total_emissions)
ax.bar_label(bars)
plt.show();